In [1]:
import pandas as pd

## Crear diccionario de FRENOS
Añadir a las órdenes de trabajo la información de su clavero

In [3]:
definiciones = pd.read_excel('data/Definiciones clave Codigo actuacion_V6.xlsx', header=2)
work_orders = pd.read_csv('data/data_ots_brake_euskotren.csv')

In [4]:
# Filtrar filas cuya columna 'clavero' empieza por 'FRE' y guardar en dataset diccionario
diccionario = definiciones[definiciones['Clavero'].astype(str).str.startswith('FRE', na=False)].copy()
diccionario.to_csv('data/diccionario.csv', index=False)

In [20]:
# Crear columna 'Código tarea std' concatenando 'clavero' y 'actuacion'
clav = work_orders['clavero'].fillna('').astype(str).str.strip()
act = work_orders['actuacion'].fillna('').astype(str).str.replace('"', '', regex=False).str.strip()
work_orders['Código tarea std'] = clav + act

In [ ]:
# Añadir a work_orders las columnas desde la 3ª columna de diccionario
cols_to_merge = list(diccionario.columns[2:4]) 

# Lookups por clave
by_code = diccionario.set_index('Código tarea std')[cols_to_merge]
by_clav = diccionario.set_index('Clavero')[cols_to_merge]

res = work_orders.copy()

# normalizar vacíos en 'actuacion' a NaN
res['actuacion'] = res['actuacion'].astype(str).str.strip()
res.loc[res['actuacion'].isin(['', '""']), 'actuacion'] = pd.NA
res['_clav_clean'] = res['clavero'].fillna('').astype(str).str.strip()

mask_act = res['actuacion'].notna()

# Inicializar columnas a NaN
for c in cols_to_merge:
    res[c] = pd.NA

# Mapeo por 'Código tarea std' cuando 'actuacion' no está vacía
map_code_dicts = {c: by_code[c].to_dict() for c in cols_to_merge}
for c, m in map_code_dicts.items():
    res.loc[mask_act, c] = res.loc[mask_act, 'Código tarea std'].map(m)

# Para filas con 'actuacion' vacía, unir por 'clavero' solo si el clavero es único en work_orders
clav_counts = diccionario['Clavero'].value_counts()
unique_clavs = set(clav_counts[clav_counts == 1].index)
print(unique_clavs)
mask_clav = (~mask_act) & res['_clav_clean'].isin(unique_clavs)
print(len(res), len(res[mask_act]), len(res[mask_clav]))

map_clav_dicts = {c: by_clav[c].to_dict() for c in cols_to_merge}
for c, m in map_clav_dicts.items():
    res.loc[mask_clav, c] = res.loc[mask_clav, '_clav_clean'].map(m)



{'FRE010503', 'FRE0604', 'FRE010113', 'FRE011207', 'FRE011005', 'FRE0702', 'FRE0605', 'FRE0307', 'FRE0803', 'FRE', 'FRE011106', 'FRE010206', 'FRE010108', 'FRE010202', 'FRE0902', 'FRE010203', 'FRE010109', 'FRE0901', 'FRE010502', 'FRE06', 'FRE010802', 'FRE0703', 'FRE0603', 'FRE0309', 'FRE080301', 'FRE0501', 'FRE011204', 'FRE080302', 'FRE010102', 'FRE0306', 'FRE11', 'FRE010808', 'FRE0606', 'FRE010704', 'FRE0705', 'FRE0706', 'FRE0502', 'FRE010111', 'FRE031203', 'FRE010406', 'FRE0304', 'FRE030104', 'FRE010905', 'FRE010510', 'FRE010602', 'FRE0704', 'FRE010710', 'FRE0305', 'FRE010713', 'FRE010402', 'FRE010303', 'FRE010906', 'FRE1104', 'FRE0701', 'FRE031201', 'FRE010902', 'FRE0906', 'FRE010606', 'FRE011205', 'FRE011301', 'FRE010403', 'FRE011002', 'FRE03', 'FRE07', 'FRE010302', 'FRE010712', 'FRE010306', 'FRE0602', 'FRE0503', 'FRE010702', 'FRE010607', 'FRE010804', 'FRE010609', 'FRE05', 'FRE011105', 'FRE011102', 'FRE0601', 'FRE0904', 'FRE010200', 'FRE0903', 'FRE011202', 'FRE0905', 'FRE0113'}
2190

In [26]:
res = res.drop(columns=['_clav_clean'])
res.to_csv('data/work_orders_dict.csv', index=False)

## Crear dataset de jerarquías
Crear un dataset con las jerarquías de los claveros para poder hacer análisis posteriores

In [6]:
def nivel1_key(clave):
    prefix = ''.join([c for c in clave if not c.isdigit()])
    nums = ''.join([c for c in clave if c.isdigit()])
    if len(nums) >= 2:
        return prefix + nums[:2]
    return None

def nivel2_key(clave):
    prefix = ''.join([c for c in clave if not c.isdigit()])
    nums = ''.join([c for c in clave if c.isdigit()])
    if len(nums) >= 4:
        return prefix + nums[:4]
    return None

In [36]:
dic= pd.read_csv('data/diccionario.csv')
dic['Clavero'] = dic['Clavero'].astype(str).str.strip()

In [37]:
dic['codigo_numerico'] = dic['Clavero'].str.replace(r'^[A-Z]+', '', regex=True)
dic['nivel'] = dic['codigo_numerico'].str.len() // 2
dic['nivel1'] = dic['Clavero'].apply(nivel1_key)
dic['nivel2'] = dic['Clavero'].apply(nivel2_key)

# Evitar duplicados en el merge: usar una única fila por 'Clavero'
dic = dic.drop_duplicates(subset='Clavero').reset_index(drop=True)

dic = dic.merge(dic[['Clavero', 'Descripción componente']], how='left',
                left_on='nivel1', right_on='Clavero', suffixes=('', '_nivel1'))
dic = dic.merge(dic[['Clavero', 'Descripción componente']], how='left',
                left_on='nivel2', right_on='Clavero', suffixes=('', '_nivel2'))

dic = dic.drop(columns=['Clavero_nivel1', 'Clavero_nivel2'])
dic = dic.rename(columns={
    'Descripción componente_nivel1': 'componente_nivel1',
    'Descripción componente_nivel2': 'componente_nivel2',
    'Clavero': 'clavero',
    'Descripción componente': 'componente'
})

In [38]:
# Hardocde de Paneles
mask_fre01 = dic['nivel'].isin([2, 3]) & (dic['nivel1'] == 'FRE01')
dic.loc[mask_fre01, 'componente_nivel1'] = 'Paneles'

# Nivel 1 → sin referencias
dic.loc[dic['nivel'] == 1, ['nivel1', 'nivel2', 'componente_nivel1', 'componente_nivel2']] = None

# Nivel 2 → mantener nivel1, quitar nivel2
dic.loc[dic['nivel'] == 2, ['nivel2', 'componente_nivel2']] = None

# Si no existe el nivel1 en el diccionario y soy nivel 2 o 3 → marcar "otros"
mask_nivel1_faltante = dic['nivel'].isin([2, 3]) & dic['componente_nivel1'].isna()
dic.loc[mask_nivel1_faltante, 'nivel1'] = None
dic.loc[mask_nivel1_faltante, 'componente_nivel1'] = 'Otros'

# Si no existe el nivel2 en el diccionario y soy nivel 3 → marcar "otros"
mask_nivel2_faltante = (dic['nivel'] == 3) & dic['componente_nivel2'].isna()
dic.loc[mask_nivel2_faltante, 'nivel2'] = None
dic.loc[mask_nivel2_faltante, 'componente_nivel2'] = 'Otros'

In [39]:
cols_keep = ['clavero', 'componente', 'nivel', 'nivel1', 'componente_nivel1', 'nivel2', 'componente_nivel2']
dic = dic[cols_keep].copy().reset_index(drop=True)
dic.to_csv("data/jerarquia.csv", index=False)